[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sergiopaniego/samples/blob/main/qwen25_omni_7B_sample.ipynb)

In [ ]:
!pip install -q -U accelerate qwen-omni-utils[decord] opencv-python datasets

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers@f742a644ca32e65758c3adb36225aef1731bd2a8

In [ ]:
!pip install flash-attn --no-build-isolation

In [ ]:
from huggingface_hub import login

login()

In [1]:
import torch
import soundfile as sf
from transformers import Qwen2_5OmniModel, Qwen2_5OmniProcessor
from qwen_omni_utils import process_mm_info

model_id = "Qwen/Qwen2.5-Omni-7B"
model = Qwen2_5OmniModel.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", attn_implementation="flash_attention_2")
processor = Qwen2_5OmniProcessor.from_pretrained(model_id)

conversation = [
    {
        "role": "system",
        "content": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.",
    },
    {
        "role": "user",
        "content": [
            # Video from: https://huggingface.co/datasets/TIGER-Lab/VideoFeedback
            {"type": "video", "video": "https://huggingface.co/datasets/hexuan21/VideoFeedback-videos-mp4/resolve/main/6/6000476.mp4"},
            {"type": "text", "text": "What's the animal en the video?"},
        ],
    },
]

USE_AUDIO_IN_VIDEO = False
# Preparation for inference
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios, images, videos = process_mm_info(conversation, use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = processor(text=text, audios=audios, images=images, videos=videos, return_tensors="pt", padding=True, use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = inputs.to(model.device).to(torch.bfloat16)

# Inference: Generation of the output text and audio
text_ids, audio = model.generate(**inputs, use_audio_in_video=USE_AUDIO_IN_VIDEO)
text = processor.batch_decode(text_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

# Output generated text and save audio
print(text)
sf.write("output.wav", audio.reshape(-1).detach().cpu().numpy(), samplerate=24000)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
Qwen2_5OmniToken2WavModel must inference with fp32, but flash_attention_2 only supports fp16 and bf16, attention implementation of Qwen2_5OmniToken2WavModel will fallback to sdpa.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

qwen-vl-utils using decord to read video.
Setting `pad_token_id` to `eos_token_id`:8292 for open-end generation.


["system\nYou are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.\nuser\nWhat's the animal en the video?\nassistant\nWell, from what I can see in the picture, it looks like an eagle. It's got that big, powerful beak and the distinctive white and dark feathers. What do you think?"]
